<a href="https://colab.research.google.com/github/FrantisekBrandejs/BAK-RQA/blob/main/BAK_RQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Využití RQA v kartografii** | bakalářská práce | František BRANDEJS, 2026

Prohlídnutí dat

In [2]:
import pandas as pd
data = pd.read_csv('drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv', sep = '\t', nrows = 10) #načtení jen prvních 10 řádků
print(data)
print(data.columns.to_list()) #výpis sloupců
#!wc -l 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' #počet řádků
#!cut -f555 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #unikátní hodnoty ve specifickém sloupci
#!cut -f19 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #co za hodnoty ma eye movement type
!cut -f18 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u

   Recording timestamp  Computer timestamp       Sensor Participant name  \
0                    0       6578398691352          NaN           Standa   
1                82154       6578398773506          NaN           Standa   
2               107157       6578398798509  Eye Tracker           Standa   
3               110490       6578398801842  Eye Tracker           Standa   
4               113823       6578398805175  Eye Tracker           Standa   
5               117157       6578398808509  Eye Tracker           Standa   
6               120490       6578398811842  Eye Tracker           Standa   
7               123823       6578398815175  Eye Tracker           Standa   
8               127157       6578398818509  Eye Tracker           Standa   
9               130490       6578398821842  Eye Tracker           Standa   

  Zkusenosti  Student KGI Pohlavi    Vek Recording name               Event  \
0      Denne          NaN     Muz  31-40     Recording1      RecordingStart   
1    

Chunk zpracování a redukce dat

In [4]:
import pandas as pd
import re
import os

file_path = 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv'
out_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

header = pd.read_csv(file_path, sep='\t', nrows=0)
all_columns = header.columns.tolist()

#Základní sloupce
base_cols = [
    'Participant name',
    'Recording timestamp',
    'Zkusenosti',
    'Eye movement type',
    'Gaze point X',
    'Gaze point Y',
    'Presented Stimulus name'
    ]

# všechny AOI sloupce (Ty co začínají AOI hit [)
aoi_cols = [c for c in all_columns if 'AOI hit [' in c]

#všechny sloupce, co se budou načítat
load_cols = base_cols + aoi_cols

#HLAVNÍ filtrace (chunk)
chunk_size = 300000 #kolik řádků se bude brát najednou
count = 0 #pro počítání, kolik řádků už je zpracováno
#pro kontrolu na konec
total_input_rows = 0
total_fixations_R = 0

print("---Začátek filtrace---")

reader = pd.read_csv(file_path, sep='\t', usecols=load_cols, chunksize=chunk_size, low_memory=False)

for i, chunk in enumerate(reader):
    total_input_rows += len(chunk) #kolik celkem vstoupí řádků
    # FILTR dle eye movement type a stimulů - pouze fixace a pouze mapy (začínající na R)
    filter_stimulus_R = chunk['Presented Stimulus name'].str.startswith('R', na=False)
    filter_fixation = chunk['Eye movement type'] == 'Fixation'

    current_fixations_count = (filter_stimulus_R & filter_fixation).sum()
    total_fixations_R += current_fixations_count

    # KOMBINACE prvních filtrů do jedné "filtery"
    final_filter = filter_stimulus_R & filter_fixation

    # FILTRACE fixací dle AOI - zatím jen připravit
    filter_legend = pd.Series(False, index=chunk.index)
    filter_all = pd.Series(False, index=chunk.index)
    filter_others_internal = pd.Series(False, index=chunk.index) # Title, Imprint, Brightest,...

    for col in aoi_cols:
        stim_in_col = col.split('[')[1].split(' -')[0] #získat z AOI hit [R9fSV - 01] pouze R9fSV
        is_hit = (chunk[col] == 1) & (chunk['Presented Stimulus name'] == stim_in_col) #musí být 1 v řádku a stimul ve sloupci se musí rovnat stimulu v řádku - to je asi trochu navíc

        if col.endswith('All]'):
            filter_all |= is_hit
        elif any(f' - 0{n}]' in col for n in range(1, 10)):
            filter_legend |= is_hit
        else:
            filter_others_internal |= is_hit

    # Fixace pouze v mapě (v All, ale ne v Legendě ani v Title/Imprint...)
    filter_pure_map = filter_all & ~filter_legend & ~filter_others_internal

    # Kopie chunku, který je omezen na jen fixace a jen stimuly začínající na R (mapy)
    reduced_chunk = chunk[final_filter].copy()

    # ROZŘAZENÍ DO KATEGORIÍ
    # všem na začátek nastavit Others
    reduced_chunk['AOI_Category'] = 'Others'

    # Zapsání nových AOI kategorií pomocí masek - .loc přesně cílí na daný řádek a sloupec
    reduced_chunk.loc[filter_pure_map[final_filter], 'AOI_Category'] = 'Map'
    reduced_chunk.loc[filter_legend[final_filter], 'AOI_Category'] = 'Legend'

    if i == 0:
        reduced_chunk[base_cols + ['AOI_Category']].to_csv(out_path, sep='\t', index=False, mode='w')
    else:
        reduced_chunk[base_cols + ['AOI_Category']].to_csv(out_path, sep='\t', index=False, mode='a', header=False)

    count += len(chunk)
    if (i + 1) % 5 == 0:
        print(f"Zpracováno {count:,} řádků...")

print("-"*30)
print("Kontrola výstupních dat:")
print(f"Celkem přečteno řádků z původního souboru: {total_input_rows:,}")
print(f"Celkem nalezeno relevantních fixací (R-stimuly): {total_fixations_R:,}")
print(f"Zahozeno řádků (sakády, jiné stimuly, šum): {total_input_rows - total_fixations_R:,}")

# Ověření přímo ze zapsaného souboru
output_count = len(pd.read_csv(out_path, sep='\t', usecols=['Participant name']))
print(f"Skutečný počet řádků v novém souboru: {output_count:,}")

if total_fixations_R == output_count:
    print("správně: Počet nalezených fixací přesně odpovídá počtu zapsaných řádků")
else:
    print("špatně: Počty nesouhlasí, zkontroluj logiku zápisu")

print("-"*30)
print(f"Data uložena v: {out_path}")
file_size_old = os.path.getsize(file_path)/10**6
file_size_new = os.path.getsize(out_path)/10**6
print(f"Velikost původního souboru: {file_size_old} MB" )
print(f"Velikost nového souboru: {file_size_new} MB")

---Začátek filtrace---
Zpracováno 1,500,000 řádků...
Zpracováno 3,000,000 řádků...


/tmp/ipython-input-4245061860.py:39: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(reader):


Zpracováno 4,500,000 řádků...
Zpracováno 6,000,000 řádků...
Zpracováno 7,500,000 řádků...
Zpracováno 9,000,000 řádků...
Zpracováno 10,209,508 řádků...
------------------------------
Kontrola výstupních dat:
Celkem přečteno řádků z původního souboru: 10,209,508
Celkem nalezeno relevantních fixací (R-stimuly): 4,746,172
Zahozeno řádků (sakády, jiné stimuly, šum): 5,463,336
Skutečný počet řádků v novém souboru: 4,746,172
správně: Počet nalezených fixací přesně odpovídá počtu zapsaných řádků
------------------------------
Data uložena v: drive/MyDrive/BAK-RQA/reduced_data.tsv
Velikost původního souboru: 8401.636338
Velikost nového souboru: 289.905463


In [6]:
out_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

import pandas as pd
test_result = pd.read_csv(out_path, sep='\t', nrows = 5)
print(test_result)
!wc -l 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

  Participant name  Recording timestamp Zkusenosti Eye movement type  \
0           Standa            295422125      Denne          Fixation   
1           Standa            295425460      Denne          Fixation   
2           Standa            295428792      Denne          Fixation   
3           Standa            295432125      Denne          Fixation   
4           Standa            295435458      Denne          Fixation   

   Gaze point X  Gaze point Y Presented Stimulus name AOI_Category  
0         905.0         321.0                   R5aJV          Map  
1         903.0         318.0                   R5aJV          Map  
2         907.0         319.0                   R5aJV          Map  
3         905.0         322.0                   R5aJV          Map  
4         904.0         319.0                   R5aJV          Map  
4746173 drive/MyDrive/BAK-RQA/reduced_data.tsv
